Effet Tunnel

In [ ]:
#import
import math as math
import numpy as np
import matplotlib as plt
print("import done")

import done
6


Fourrier transform

In [ ]:
def g(x):
    g=np.exp(-x**2)
    return g

def mu_ech(f):
    k=1000 #nombre d'instances
    T=0.1
    dT=0.2 # doit être inférieur à T
    for k in range:
        dTe

def F(f,mu):
    k0=1000000
    mu_ech=1
    sum=0
    for k in range (-k0,k0+1):
        sum=sum+f(k)*np.exp(-2*1j*np.pi*k*mu/mu_ech)
    return sum

print(F(g,4))
print(g(4))

(9.597878047884478e-14+4.260811287071875e-07j)
-0.7568024953079282
